# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022


# <span style="color:#006E7F">__TP2 - Assembly and correction__ <a class="anchor" id="data"></span>  
    
## <span style="color: #4CACBC;"> 1. Assemblies </span>  

More contiguous genome assemblies can be generated using long sequencing read but assembly is not a quiet pace. Eukaryotic genomes assembly is a complex task (large genome sizes, high rates of repeated sequences, high heterozygosity levels and even polyploidy). While prokaryotic genomes may appear less challenging, specific features such as circular DNA molecules, must be taken into consideration to achieve high quality assembly.

* For assembly, ONT recommend sequencing a human genome to a minimum depth of 30x of 25–35 kb reads.
However, sequencing to a depth of 60x is advisable to obtain the best assembly metrics. We also recommend basecalling in high accuracy mode. Greatest contig N50 is usually obtained with Shasta and Flye. Polishing/Correction is also recommended (Racon and Medaka). https://nanoporetech.com/sites/default/files/s3/literature/human-genome-assembly-workflow.pdf


* Long reads simplify genome assembly, with the ability to span repeat-rich sequences (characteristic of  antimicrobial resistance genes) and structural variants. Nanopore sequencing also shows a lack of
bias in GC-rich regions, in contrast to other sequencing platforms. To perform microbial genome assembly, we suggest using the third-party de novo assembly tool Flye. We also recommend one round of polishing with Medaka. https://nanoporetech.com/sites/default/files/s3/literature/microbial-genome-assembly-workflow.pdf 

Flye  https://github.com/fenderglass/Flye 

Canu  https://canu.readthedocs.io/en/latest/quick-start.html

Miniasm  https://github.com/lh3/miniasm + Minipolish version https://github.com/rrwick/Minipolish

Shasta  https://github.com/chanzuckerberg/shasta

Smartdenovo  https://github.com/ruanjue/smartdenovo

Raven  https://github.com/lbcb-sci/raven

### <span style="color: #4CACBD;"> 1.1 Assembly using Flye </span>

We are going to assembly some Clones by using Flye https://github.com/fenderglass/Flye

Flye generates the concatenation of multiple disjoint genomic segments called disjointigs to build a repeat graph. Reads are mapped to this repeat graph to resolve conflicts (unbridged repeats) and output contigs.

In [ ]:
# declare your clone number
CLONE="Clone10"
# create a repertory for save flye assembly results
cd ~/work/RESULTS/
pwd

In [ ]:
# Run flye
time flye --nano-raw ../DATA/${CLONE}/ONT/${CLONE}.fastq.gz --genome-size 1000000 --out-dir FLYE --threads 4

### <span style="color: #4CACBD;"> 1.2 Assembly using Raven </span>

In [ ]:
# create a repertory for save raven assembly results
mkdir ~/work/RESULTS/RAVEN
cd ~/work/RESULTS/RAVEN
pwd

In [ ]:
# Run raven
time raven -p 0 --graphical-fragment-assembly ${CLONE}_raven.gfa -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz > ${CLONE}_raven.fasta

* How many contigs were obtained by Flye and Raven ? Please fill in results in the shared [file](https://lite.framacalc.org/9pd3-ont_sg_2021)
* Calculate first statistics about assemblies: N50 and lenght contig mean.

In [ ]:
grep -c ^'>' ~/work/RESULTS/*/*.fasta

## <span style="color: #4CACBC;"> 2. Polishing assemblies with Racon </span>  

Racon corrects raw contigs generated by rapid assembly methods with original ONT reads.

From 2 to 4 racon rounds are usually used by the community. 

Polish contigs assembled by Flye using two rounds of racon !.

In [ ]:
racon

## Racon first round

In [ ]:
mkdir -p ~/work/RESULTS/FLYE_RACON
cd ~/work/RESULTS/FLYE_RACON
time minimap2 -t 6 ../FLYE/assembly.fasta ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz 1> assembly.minimap4racon1.paf
time racon -t 6 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz assembly.minimap4racon1.paf ../FLYE/assembly.fasta > assembly.racon1.fasta

## Racon second round

In [ ]:
# second round
cd ~/work/RESULTS/FLYE_RACON
time minimap2 -t 4 assembly.racon1.fasta ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz 1> assembly.minimap4racon2.paf
time racon -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz assembly.minimap4racon2.paf assembly.racon1.fasta > assembly.racon2.fasta

## <span style="color: #4CACBC;"> 3. Assemblies correction with Medaka </span>  

Correction can improve the consensus sequence for a draft genome assembly.

Medaka uses fast5 files to correct contigs using trained models. These models are freely available.

Medaka allows you to train a model by using fast5 from your favorite species. You can use it directly to obtain a consensus from you favorite organism.

### <span style="color: #4CACBC;"> 1.3 Correct assemblies with Medaka </span>  

### <span style="color: #4CACBC;"> 1.3.1 Before to correct assemblies, index them </span>  

In [ ]:
cd ~/work/RESULTS/FLYE_RACON
pwd

In [ ]:
time samtools faidx assembly.racon2.fasta 
time minimap2 -d assembly.racon2.fasta.mmi assembly.racon2.fasta

In [ ]:
#create a medaka repertory
mkdir -p ~/work/RESULTS/FLYE_RACON_MEDAKA
cd ~/work/RESULTS/FLYE_RACON_MEDAKA
pwd

### <span style="color: #4CACBC;"> 1.3.2 Medaka_consensus </span>  


Medaka is a tool to create a consensus sequence from nanopore sequencing data. 

This task is performed by using neural networks applied to a pileup of individual sequencing reads against a draft assembly.

It outperforms graph-based methods operating on basecalled data, and can be competitive with state-of-the-art signal-based methods whilst being much faster.

As input medaka accepts reads in either a .fasta or a .fastq file. It requires a draft assembly as a .fasta.

### Check the usage of medaka_consensus

In [ ]:
medaka_consensus

### Check the medaka model to use

Medaka models are named to indicate i) the pore type, ii) the sequencing device (MinION or PromethION), iii) the basecaller variant, and  iv) the basecaller version

{pore}_{device}_{caller variant}_{caller version}

examples: 

r941_min_fast_g303 : MiniON R9.4.1 flowcells using the fast Guppy basecaller version 3.0.3. 


In [ ]:
medaka tools list_models

### Medaka in FLYE + RACON

We run medaka on the FLYE + polished with racon assembly.

In [ ]:
# go to medaka repertory results
cd ~/work/RESULTS/FLYE_RACON_MEDAKA

In [ ]:
# activate medaka conda environement
conda activate medaka

In [ ]:
# medaka_consensus -i ${BASECALLS} -d ${DRAFT} -o ${OUTDIR} -t ${NPROC} -m r941_min_high_g360
time medaka_consensus -i ~/work/DATA/${CLONE}/ONT/${CLONE}.fastq.gz -d ~/work/RESULTS/FLYE_RACON/assembly.racon2.fasta -o MEDAKA_CONSENSUS -t 4 -m r941_min_high_g360

In [ ]:
conda deactivate

### <span style="color: #4CACBC;"> Conclusion </span>  


1. Here, we have obtained a pseudomolecule of a clone by using Flye.

2. We have polished it twice with racon.

3. We have corrected it with medaka models.

You can do similar pipeline using the RAVEN assembler with your favorite Clone.

We will compare the results in next practical "Quality Assemblies". 